In [2]:
cd "/home/sage/Documents/works/12. Developments/07. sage/02. IdIso/"

/home/sage/Documents/works/12. Developments/07. sage/02. IdIso


#  <font color="green">Implementation of IdealToIsogeny algorithm</font>

## <font color="green">0. Prime Search</font>

In [3]:
load('primegen.sage')
find_prime(20, 5)

n: 20
n: 21
n: 22
n: 23
n: 24
n: 25
n: 26
n: 27
n: 28
n: 29
n: 30
n: 31
n: 32
n: 33
n: 34
n: 35
n: 36
n: 37
n: 38
n: 39
n: 40
n: 41
n: 42
n: 43
n: 44
n: 45
n: 46
n: 47
n: 48
n: 49
n: 50
n: 51
n: 52
n: 53
n: 54
n: 55
n: 56
n: 57
n: 58
n: 59
n: 60
n: 61
n: 62
n: 63
n: 64
n: 65
41320706725109395619839 is not prime
n: 66
41320706725109395619839 is not prime
n: 67
41320706725109395619839 is not prime
n: 68
41320706725109395619839 is not prime
n: 69
41320706725109395619839 is not prime
n: 70
41320706725109395619839 is not prime
n: 71
n: 72
n: 73
n: 74
n: 75
4344577164240073596600319 is not prime
n: 76
n: 77
n: 78
n: 79
142048783804718928027975679 is not prime
n: 80
n: 81
n: 82
n: 83
n: 84
n: 85
n: 86
n: 87
n: 88
n: 89
n: 90
n: 91
n: 92
n: 93
n: 94
n: 95
n: 96
n: 97
3588243480271031849611605510717439 is not prime
n: 98
n: 99
n: 100
n: 101
n: 102
n: 103
n: 104
n: 105
n: 106
n: 107
n: 108
2576677316047908210658258803375472639 is not prime
n: 109
2576677316047908210658258803375472639 is not prim

KeyboardInterrupt: 

In [4]:
prime = 393108616061618412489735027422936257282820381081599
Fp = GF(prime)
Fp2.<Fp_i> = GF(prime^2, modulus = x^2 + 1, impl='pari_ffelt')
R.<x> = PolynomialRing(Fp)
print((x^2 + 1).factor())
# Fp4.<Fp_j> = GF(prime^4, modulus = x^4 - 6*x^2 + 7)


KeyboardInterrupt: 

## <font color="green">1. Prerequisites</font>

### Load required libraries

In [74]:
load('quaternion_tools.sage')
load('isogeny.sage')
load('richelot.sage')
load('const.sage')

### Cornaccia algorithm

In [75]:
def cornacchia(d, m):
    try: 
        (x,y) = DiagonalQuadraticForm(QQ, [1,d]).solve(m)
        if not x.is_integer() or not y.is_integer() : raise Exception("No solution")
        return x, y
    except: raise Exception("No solution")

## <font color="green">2. Setting the lab environment</font>

### 1. Get a random ideal $I$ of norm $\ell^e$

In [76]:
I = GetRandomIdealElle()
assert(I.norm() == ell^6)

### 2. Extract the first ideal factor $I_1$
+ $I = I_1 \cdot I_2 \cdot ... \cdot I_k$ and $n(I_i) = \ell$

In [77]:
ellO0 = QA.ideal(O0.basis()).scale(ell)
I1 = AddIdeal(I, ellO0)

### 3. Compute corresponding isogeny $\phi_1$ to $I_1$
+ $ker(\phi_1) = ker(I_1)$

In [78]:
phi1_kernel_generator = left_O0_ideal_to_isogeny_kernel(I1)
phi1 = Isogeny(Fp(0), phi1_kernel_generator)
assert(I1.norm() == ell)

### 4. Get the codomain curve $E_1$ of $\phi_1$ and the dual isogeny $\hat{\phi}_1$

In [142]:
E1 = phi1.codomain_curve
phi1_dual = phi1.dual_isogeny()

### 5. Compute the right order $\mathcal{O}_1$ of $I_1$
+ $\mathcal{O}_1 \cong End(E_1)$

In [80]:
O1 = I1.right_order()

### 6. Extract the second ideal factor $I_2$
+ $\bar{I_1} I \cap \ell \mathcal{O}_1 = I_2$

In [81]:
I2 = I1.conjugate() * I
I2 = I2.scale(1/ell)
ellO1 = QA.ideal(O1.basis()).scale(ell)
I2 = AddIdeal(I2, ellO1)

assert(I2.norm() == ell and I2.left_order() == O1)

## <font color="green">3. Experiments</font>

+ Compute the generator $\alpha_2$ of $I_2$ such that $I_2 = \alpha_2 \mathcal{O_1} + \ell \mathcal{O_1}$
+ Hopefully $n(\alpha_2)$ is as small as possible
+ Note that $\alpha_2 \in I_2 \subset \mathcal{O}_1$

In [82]:
I2_generator = ideal_generator(I2)

+ Compute the reduced basis $\{o_{11}, o_{12}, o_{13}, o_{14}\}$ of $\mathcal{O_1}$

In [83]:
O1_basis = GetReducedBasis(O1.basis())

+ Try to find the $\beta \in \mathcal{O}_1$ such that $n(\alpha_2) + n(\beta) = 2^{e_1}\cdot 3^{e_2} | \sqrt{\#E_1(\mathbb{F}_{p^k})}$
+ $\mathrm{ord}_{2}(\#E_1(\mathbb{F}_{p^k})) \approx \mathrm{ord}_2(\#E_1(\mathbb{F}_{p^2}))=2 \times \mathrm{ord}_2(p+1)$ regardless of the extension size $k$
    + So it must be satisfied that $2^e < (p+1)$
+ Find the coefficients $\{c_1, c_2, c_3, c_4\}$ such that $n(\beta) = n(\sum_{i}{c_i o_{1i}}) = 2^e - n(\alpha_2)$

In [84]:
import random
I2_basis = GetReducedBasis(I2.basis())
I2_generator = I2_basis[2]
e1 = 148
e2 = 16
gram = matrix([[1,0,0,0],[0,1,0,0],[0,0,prime,0],[0,0,0,prime]])
M_O1 = matrix([vector(v.coefficient_tuple()) for v in O1_basis])
G = M_O1 * gram * M_O1.transpose()

x,y = 0, 0
k = 100
for i in range(k^2):
    I2_gen = I2_generator + random.randint(-k, k) * I2_basis[1] + random.randint(-k, k) * I2_basis[0]
    if gcd(I2_gen.reduced_norm() ,6) != 1: continue
    target = 2^e1*3^e2 - I2_gen.reduced_norm()
    if kronecker(-G[1][1], target) != 1: continue
    try:
        x,y = cornacchia(G[1][1], target)
        break
    except:
        print("no solution of cornacchia algorithm")
beta = O1_basis[0]*x + O1_basis[1]*y
print((beta.reduced_norm() + I2_gen.reduced_norm()).factor())

no solution of cornacchia algorithm
no solution of cornacchia algorithm
no solution of cornacchia algorithm
no solution of cornacchia algorithm
no solution of cornacchia algorithm
no solution of cornacchia algorithm
no solution of cornacchia algorithm
no solution of cornacchia algorithm
no solution of cornacchia algorithm
no solution of cornacchia algorithm
no solution of cornacchia algorithm
no solution of cornacchia algorithm
no solution of cornacchia algorithm
no solution of cornacchia algorithm
no solution of cornacchia algorithm
no solution of cornacchia algorithm
no solution of cornacchia algorithm
no solution of cornacchia algorithm
no solution of cornacchia algorithm
no solution of cornacchia algorithm
no solution of cornacchia algorithm
no solution of cornacchia algorithm
no solution of cornacchia algorithm
no solution of cornacchia algorithm
no solution of cornacchia algorithm
no solution of cornacchia algorithm
no solution of cornacchia algorithm
no solution of cornacchia al

In [156]:
N = 2^e1*3^e2
N_alpha = I2_gen.reduced_norm()
N_beta = beta.reduced_norm()
x_value = Integer(-mod(N_alpha, N) * mod(N_beta, N)^-1)

In [86]:
P1, Q1 = phi1.codomain_curve.gens()
cofactor = Integer((prime^2 - 1) / N)
P1, Q1 = cofactor*P1, cofactor*Q1
E1.is_on_curve(P1[0], P1[1])

True

In [152]:
load('isogeny.sage')
T1 = eval_endomorphism(O1(QA_j), P1, phi1, phi1_dual)
T2 = eval_endomorphism(O1(QA_j), T1, phi1, phi1_dual)
print(T2)
print(P1*prime)

(0, -5, 10, 0)
(0, -5, 10, 0)
(129113453939286971135777935728296728698062056617913*Fp2_i^3 + 26317246708750674447876042223891343990981391721820*Fp2_i^2 + 304254845974847938480599976457466215547954908525976*Fp2_i + 48303859068627644946929280296033193216449845016722 : 128653902677626437719859671508649052114840034694463*Fp2_i^3 + 226220870735259432206616764909889054146993948259910*Fp2_i^2 + 84498824648579456009498265116702823437186124362880*Fp2_i + 246929716029581740127794717055169869566031471378649 : 1)
(129113453939286971135777935728296728698062056617913*Fp2_i^3 + 26317246708750674447876042223891343990981391721820*Fp2_i^2 + 304254845974847938480599976457466215547954908525976*Fp2_i + 48303859068627644946929280296033193216449845016722 : 264454713383991974769875355914287205167980346387136*Fp2_i^3 + 166887745326358980283118262513047203135826432821689*Fp2_i^2 + 308609791413038956480236762306233433845634256718719*Fp2_i + 146178900032036672361940310367766387716788909702950 : 1)


In [147]:
O0_coordinate(beta)

(35216418175516094286375649, -118808485013396080296513420, 0, 0)

In [149]:
load('isogeny.sage')
alpha_P1 = eval_endomorphism(I2_gen, P1, phi1, phi1_dual)
alpha_Q1 = eval_endomorphism(I2_gen, Q1, phi1, phi1_dual)

(682, -1987, 0, 1)
(682, -1987, 0, 1)


In [159]:
beta_P1 = eval_endomorphism(beta, P1, phi1, phi1_dual)
beta_Q1 = eval_endomorphism(beta, Q1, phi1, phi1_dual)

(176082090877580471431878245, -594042425066980401482567100, 0, 0)
(176082090877580471431878245, -594042425066980401482567100, 0, 0)


In [196]:
points = [alpha_P1, alpha_Q1, Integer(x_value)*beta_P1, Integer(x_value)*beta_Q1]
points = [E1((e[0], e[1])) for e in points]
assert points[0].weil_pairing(points[1], N)*points[2].weil_pairing(points[3], N) == 1


In [197]:
load('../01. jacobian_isogeny/22_isogeny.sage')
kernel = [(points[0], points[2]), (points[1], points[3])]
a, b = 148, 16
kernel2 = [(3^b*2^(a-1)*D[0], 3^b*2^(a-1)*D[1]) for D in kernel]
E_start = E1
h, points, isog = Gluing22(E_start, E_start, kernel2, eval_points = kernel)

In [198]:
kernel22 = [3^b * 2 * D for D in points]
j, points = isogeny_22(kernel22, points, a-2)

In [195]:
load('../01. jacobian_isogeny/33_isogeny.sage')
kernel33 = [2 * D for D in points]
K = Fp
j, points = isogeny_33(kernel33, points, b)

Interrupting Singular...


KeyboardInterrupt: Restarting Singular (WARNING: all variables defined in previous session are now invalid)

In [191]:
G1 = points[0][0]
G2 = points[1][0]
h = j.curve().hyperelliptic_polynomials()[0]
G3, r3 = h.quo_rem(G1*G2)
assert r3 == 0
delta = Matrix(G.padded_list(3) for G in (G1, G2, G3))
if delta.determinant() : print("error")

error


In [190]:
G3.padded_list(3)

[216444075941163185554446526284284324406228296974745*Fp2_i^2 + 256513331552043670183370824271572942448297678523416,
 260471929515966099235867624627797553078549135673902*Fp2_i^2 + 56104692588624402979074611396624832003443150188107,
 237144324763996971113766813297928442412645365533273*Fp2_i^2 + 185562636951925495015311973323764388739463843052667]

In [186]:
2*points[1]

(1)